In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import gc
import os
import json
import math
import sys
import time

import numpy as np
import tensorflow as tf
import scipy.sparse as sp
import faiss
import implicit

import data
import modeling

# 1. Find the Dataset

I use one of the datasets from https://arxiv.org/pdf/2005.09347.pdf. It is the Amazon Review Data (2018). The dataset is described: 

"""This Dataset is an updated version of the Amazon review dataset released in 2014. As in the previous version, this dataset includes reviews (ratings, text, helpfulness votes), product metadata (descriptions, category information, price, brand, and image features), and links (also viewed/also bought graphs)."""

I use the data for the Books category, the category where it has all started.

I pick the dataset based on the following pros:
- The dataset clearly presents the sequential interaction of users with the products.
- The dataset contains real-world interactions for the largest e-commerce store in the world.
- It is complex as there are hundreds of thousands of users and items (the exact number of users is 482934, the exact number of items is 367964.

In [4]:
train_name = "train"
valid_name = "valid"
test_name = "test"

In [5]:
data_path = "/data/msazanovich/recommender-systems/data/"
batch_size = 128
maxlen = 20

In [6]:
dataset = "book"
train_file = os.path.join(data_path, "book_data", f"{dataset}_train.txt")
valid_file = os.path.join(data_path, "book_data", f"{dataset}_valid.txt")
test_file = os.path.join(data_path, "book_data", f"{dataset}_test.txt")

In [7]:
def summarize_data_split(data_iter, split):
  print(f'{split} users:', len(data_iter.users))
  print(f'{split} items:', len(data_iter.items))
  interactions = 0
  for user, items in data_iter.graph.items():
    interactions += len(items)
  print(f'{split} interactions:', interactions)

In [8]:
train_data = data.DataIterator(train_file, batch_size, maxlen, is_train=True)

In [9]:
user_count = max(train_data.users) + 1
item_count = max(train_data.items) + 1

In [10]:
summarize_data_split(train_data, 'Train')

Train users: 482934
Train items: 367964
Train interactions: 7145481


In [11]:
valid_data = data.DataIterator(valid_file, batch_size, maxlen, is_train=False)

In [12]:
summarize_data_split(valid_data, 'Valid')

Valid users: 60367
Valid items: 249664
Valid interactions: 874813


In [13]:
test_data = data.DataIterator(test_file, batch_size, maxlen, is_train=False)

In [14]:
summarize_data_split(test_data, 'Test')

Test users: 60367
Test items: 249724
Test interactions: 877747


In [15]:
item_asin_to_id = {}
with open(os.path.join(data_path, "book_data", "book_item_map.txt"), "r") as f:
  for line in f:
    item_asin, item_id = line.strip().split(',')
    item_asin_to_id[item_asin] = int(item_id)

In [16]:
item_id_to_title = {}
with open(os.path.join(data_path, "meta_Books.json"), "r") as f:
  for line in f:
    r = eval(line.strip())
    item_asin = r["asin"]
    if item_asin in item_asin_to_id:
      item_id = item_asin_to_id[item_asin]
      item_title = r.get("title", "N/A")
      item_id_to_title[item_id] = item_title

# 1.5 Setup the Evaluation method

In [17]:
def build_item_index(item_embs):
  res = faiss.StandardGpuResources()
  flat_config = faiss.GpuIndexFlatConfig()
  flat_config.device = 0
  gpu_index = faiss.GpuIndexFlatIP(res, embedding_dim, flat_config)
  gpu_index.add(item_embs)
  return gpu_index


def eval_model_on(data_iter, item_embs_fn, user_embs_fn, top_n=50):
  item_embs = item_embs_fn()
  gpu_index = build_item_index(item_embs)
  
  start_time = time.time()

  total = 0
  recall_sum = 0.0
  ndcg_sum = 0.0
  hitrate_sum = 0.0
  for src, tgt in data_iter:
    (user_id_list, item_id_list), (hist_item_list, hist_mask_list) = src, tgt
    user_embs = user_embs_fn(hist_item_list, hist_mask_list)
    D, I = gpu_index.search(user_embs, top_n)
    for i, relevant_ids in enumerate(item_id_list):
      recall = 0
      dcg = 0.0
      for j in range(top_n):
        if I[i][j] in relevant_ids:
          recall += 1
          dcg += 1.0 / math.log(j+2, 2)
      idcg = 0.0
      for j in range(min(len(relevant_ids), top_n)):
        idcg += 1.0 / math.log(j+2, 2)

      total += 1
      recall_sum += recall * 1.0 / len(relevant_ids)
      ndcg_sum += dcg / idcg
      hitrate_sum += 1.0 if recall > 0 else 0.0

  recall = recall_sum / total
  ndcg = ndcg_sum / total
  hitrate = hitrate_sum / total
  return {'users': total, 'recall': recall, 'ndcg': ndcg, 'hitrate': hitrate}

In [18]:
model_metrics = {}

In [19]:
eval_item_id = list(range(1, 10))

In [20]:
def eval_similar_items(item_embs_fn):
  item_embs = item_embs_fn()
  gpu_index = build_item_index(item_embs)

  eval_item_embs = [item_embs[item_id] for item_id in eval_item_id]
  D, I = gpu_index.search(np.array(eval_item_embs), 10)
  for item_id, ii, dd in zip(eval_item_id, D, I):
    print("Item title:", item_id_to_title[item_id])
    print("Similar items are:")
    for d, i in zip(ii, dd):
      print(d, item_id_to_title[i])
    print()

In [21]:
(_, _), (eval_hist_item_list, eval_hist_mask_list) = next(test_data)
eval_hist_item_list = eval_hist_item_list[:10]
eval_hist_mask_list = eval_hist_mask_list[:10]
test_data.index = 0

In [22]:
def eval_user_recommendations(item_embs_fn, user_embs_fn):
  item_embs = item_embs_fn()
  gpu_index = build_item_index(item_embs)
  
  user_embs = user_embs_fn(eval_hist_item_list, eval_hist_mask_list)
  D, I = gpu_index.search(user_embs, 10)
  for hist_item_list, hist_mask_list, dd, ii in zip(eval_hist_item_list, eval_hist_mask_list, D, I):
    print('For the user with history:')
    for item, mask in zip(hist_item_list, hist_mask_list):
      if mask == 0.0:
        break
      print(item_id_to_title[item])
    print("Recommendations are are:")
    for d, i in zip(dd, ii):
      print(d, item_id_to_title[i])
    print()

# 2. Collaborative Filtering

I choose the Bayesian Personalized Ranking (BPR) for the collaborative filtering model.

In [23]:
users_coo = []
items_coo = []
for user, items in train_data.graph.items():
  users_coo.extend([user] * len(items))
  items_coo.extend(items)
print(len(users_coo))
print(len(items_coo))

user_item = sp.coo_matrix((np.ones_like(users_coo), (users_coo, items_coo)))
user_item_t_csr = user_item.T.tocsr()

del users_coo, items_coo
gc.collect()

7145481
7145481


0

In [24]:
embedding_dim = 64
bpr_iterations = 1000
model_bpr = (
  implicit.bpr.BayesianPersonalizedRanking(
    factors=embedding_dim,
    iterations=bpr_iterations,
    use_gpu=False))

In [25]:
model_bpr.fit(user_item_t_csr)

In [26]:
bpr_item_embs = model_bpr.item_factors[:, :-1]
bpr_item_embs = np.ascontiguousarray(bpr_item_embs)
bpr_item_embs.shape

(367983, 64)

In [27]:
def bpr_item_embs_fn():
  return bpr_item_embs

In [28]:
def bpr_user_embs_fn(hist_item_list, hist_mask_list):
  hist_item_arr = np.array(hist_item_list, dtype=np.int32)
  hist_mask_arr = np.array(hist_mask_list, dtype=np.float32)
  n, m = hist_mask_arr.shape
  hist_item_embs = bpr_item_embs[hist_item_arr.flatten()].reshape((n, m, -1))
  hist_item_embs = hist_item_embs * np.expand_dims(hist_mask_arr, axis=-1)
  user_embs = np.sum(hist_item_embs, axis=1) / np.sum(hist_mask_arr, axis=1, keepdims=True)
  return user_embs

## CF Quantitative evaluation 

In [29]:
test_metrics = eval_model_on(
    data_iter=test_data,
    item_embs_fn=bpr_item_embs_fn,
    user_embs_fn=bpr_user_embs_fn)
print(f"test metrics: {test_metrics}")

test metrics: {'users': 60367, 'recall': 0.0793839708971102, 'ndcg': 0.030340466471052048, 'hitrate': 0.157287922209154}


In [30]:
model_metrics["BPR"] = test_metrics

## CF Qualitative evaluation

In [31]:
eval_similar_items(bpr_item_embs_fn)

Item title: Gone Girl
Similar items are:
4.8599844 Gone Girl
3.9932723 Sharp Objects: A Novel
3.885926 Dark Places: A Novel
3.6470726 Defending Jacob: A Novel
3.6303177 The Silent Wife: A Novel
3.4425368 The Dinner
3.3613777 The Husband's Secret
3.2325141 The Light Between Oceans
3.223132 Tell the Wolves I'm Home: A Novel
3.1953988 What We Leave Behind

Item title: The Hunger Games (The Hunger Games, Book 1)
Similar items are:
5.520539 The Hunger Games (The Hunger Games, Book 1)
5.338859 Mockingjay (The Final Book of The Hunger Games)
5.1231794 Catching Fire (The Hunger Games, Book 2)
5.0399427 Catching Fire (The Hunger Games, Book 2)
3.5671346 Dream Cat
3.2384362 Catching Fire
3.1875901 The Hunger Games: Official Illustrated Movie Companion
3.1383018 Class A (CHERUB #2) (Bk. 2)
3.1333385 Katniss the Cattail: An Unauthorized Guide to Names and Symbols in Suzanne Collins' The Hunger Games
3.125792 The Hunger Games Tribute Guide

Item title: The Book Thief
Similar items are:
5.1500897 Th

In [32]:
eval_user_recommendations(bpr_item_embs_fn, bpr_user_embs_fn)

For the user with history:
Gone Girl
The Dog Stars (Vintage Contemporaries)
Fahrenheit 451 Low Price CD
Ready Player One
Recommendations are are:
2.737611 The Dog Stars (Vintage Contemporaries)
2.5838308 Ready Player One
2.4328501 Age of Miracles, the (Lib)(CD)
2.4078915 The Fifty Year Sword
2.3656206 Robopocalypse: A Novel (Vintage Contemporaries)
2.2266612 Invisible Monsters Remix
2.2078996 The Mirage: A Novel
2.1832147 The Year of the Flood
2.167082 The Collective: A Novel
2.156073 The Last Policeman

For the user with history:
One Last Thing Before I Go: A Novel
Sacre Bleu: A Comedy d'Art
Fahrenheit 451 Low Price CD
The Time Keeper
How to Talk to a Widower
The Adventures of Sherlock Holmes (Dover Thrift Editions)
The Casual Vacancy
Brain on Fire: My Month of Madness
Unbroken
The Martian Chronicles (Harper Perennial Modern Classics)
The Twelve Tribes of Hattie (Vintage)
The Round House
When She Woke
Mr. Penumbra's 24-Hour Bookstore: A Novel
The Elegance of the Hedgehog
The Bungalow:

# 3. Neural Collaborative Filtering

For the Neural Collaborative Filtering model, I use the model where the user embedding is not just an average of the items from the recent history, but it is also mapped through the trainable dense layer to the hidden layer (the size of which is equal to the embedding size).

The user embedding is then used to optimize the inner product with the item following in history. The optimization is done through sampled (since there are 367964 items) softmax loss for the correct item along with negative samples.

In [33]:
embedding_dim = 64
hidden_size = 64
lr = 0.005

gpu_options = tf.GPUOptions(allow_growth=True)

sess_dnn = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
model_dnn = modeling.Model_DNN(item_count, embedding_dim, hidden_size, batch_size, maxlen)

sess_dnn.run(tf.global_variables_initializer())
sess_dnn.run(tf.local_variables_initializer())



Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use keras.layers.dense instead.



In [34]:
def dnn_item_embs_fn():
  return model_dnn.output_item(sess_dnn)

In [35]:
def dnn_user_embs_fn(hist_item_list, hist_mask_list):
  user_embs = model_dnn.output_user(sess_dnn, [hist_item_list, hist_mask_list])
  return user_embs

In [36]:
start_time = time.time()

max_iters = 20000
test_iter = 1000
valid_best_recall = 0.0
valid_best_iter = 0
train_data_iter = iter(train_data)

loss_sum = 0.0
for iters in range(1, max_iters + 1):
  src, tgt = next(train_data_iter)
  (user_id_list, item_id_list), (hist_item_list, hist_mask_list) = src, tgt
  loss = model_dnn.train(sess_dnn, [user_id_list, item_id_list, hist_item_list, hist_mask_list, lr])
  loss_sum += loss

  if iters % test_iter == 0:
    test_time = time.time()
    valid_metrics = eval_model_on(
        data_iter=valid_data,
        item_embs_fn=dnn_item_embs_fn,
        user_embs_fn=dnn_user_embs_fn)
    print(f"iters: {iters}")
    print(f"train loss: {loss_sum / test_iter}")
    print(f"valid metrics: {valid_metrics}")
    print(f"time spent so far: {((time.time() - start_time) / 60.0):.3f}m")
    loss_sum = 0.0
    
    valid_recall = valid_metrics["recall"]
    if valid_recall > valid_best_recall:
      valid_best_recall = valid_recall
      valid_best_iter = iters
    
    if iters - valid_best_iter >= 5 * test_iter:
      print('early stopping criterion is met')
      break

iters: 1000
train loss: 7.588786633968353
valid metrics: {'users': 60367, 'recall': 0.03216566536780674, 'ndcg': 0.013776011606282003, 'hitrate': 0.07330163831232296}
time spent so far: 1.522m
iters: 2000
train loss: 6.906737147331238
valid metrics: {'users': 60367, 'recall': 0.04243955138380641, 'ndcg': 0.018304352188938375, 'hitrate': 0.09390892374973081}
time spent so far: 3.045m
iters: 3000
train loss: 6.39157055568695
valid metrics: {'users': 60367, 'recall': 0.05105718947374024, 'ndcg': 0.02220283882575626, 'hitrate': 0.11330693922176023}
time spent so far: 4.569m
iters: 4000
train loss: 5.953585556507111
valid metrics: {'users': 60367, 'recall': 0.05474238606220706, 'ndcg': 0.024337278849042195, 'hitrate': 0.12115891132572432}
time spent so far: 6.093m
iters: 5000
train loss: 5.605281569004059
valid metrics: {'users': 60367, 'recall': 0.05916667184388413, 'ndcg': 0.026564492986180445, 'hitrate': 0.1285636191959183}
time spent so far: 7.618m
iters: 6000
train loss: 5.306699497699

## NCF Quantitative evaluation 

In [37]:
test_metrics = eval_model_on(
    data_iter=test_data,
    item_embs_fn=dnn_item_embs_fn,
    user_embs_fn=dnn_user_embs_fn)
print(f"test metrics: {test_metrics}")

test metrics: {'users': 60367, 'recall': 0.06991044218893151, 'ndcg': 0.03219544022900266, 'hitrate': 0.15322941342124008}


In [38]:
model_metrics['DNN'] = test_metrics

## NCF Qualitative evaluation

In [39]:
eval_similar_items(dnn_item_embs_fn)

Item title: Gone Girl
Similar items are:
33.14178 Gone Girl
28.836876 Inferno
27.388721 The Hunger Games (The Hunger Games, Book 1)
27.272175 Sycamore Row
27.162636 The Book Thief
27.100647 Heaven is for Real Movie Edition: A Little Boy's Astounding Story of His Trip to Heaven and Back
25.947153 Doctor Sleep: A Novel
25.860832 The Goldfinch: A Novel (Pulitzer Prize for Fiction)
25.31991 Allegiant (Divergent, #3)
24.784689 Fifty Shades of Grey: Book One of the Fifty Shades Trilogy

Item title: The Hunger Games (The Hunger Games, Book 1)
Similar items are:
37.68365 The Hunger Games (The Hunger Games, Book 1)
29.933704 Inferno
27.731993 The Book Thief
27.72144 Heaven is for Real Movie Edition: A Little Boy's Astounding Story of His Trip to Heaven and Back
27.388721 Gone Girl
26.577408 Fifty Shades of Grey: Book One of the Fifty Shades Trilogy
26.396591 Allegiant (Divergent, #3)
25.734365 Under The Dome: A Novel
25.57756 Insurgent (Divergent)
25.371483 Unbroken

Item title: The Book Thief


In [40]:
eval_user_recommendations(dnn_item_embs_fn, dnn_user_embs_fn)

For the user with history:
Gone Girl
The Dog Stars (Vintage Contemporaries)
Fahrenheit 451 Low Price CD
Ready Player One
Recommendations are are:
12.158984 The Book Thief
11.356752 Ready Player One
11.200513 The Casual Vacancy
11.031932 Fahrenheit 451 Low Price CD
11.010069 Life After Life: A Novel
10.998772 2Br02B
10.634503 Where'd You Go, Bernadette: A Novel
10.631555 Zealot: The Life and Times of Jesus of Nazareth
10.593879 Red Rising
10.488182 Bad Monkey

For the user with history:
One Last Thing Before I Go: A Novel
Sacre Bleu: A Comedy d'Art
Fahrenheit 451 Low Price CD
The Time Keeper
How to Talk to a Widower
The Adventures of Sherlock Holmes (Dover Thrift Editions)
The Casual Vacancy
Brain on Fire: My Month of Madness
Unbroken
The Martian Chronicles (Harper Perennial Modern Classics)
The Twelve Tribes of Hattie (Vintage)
The Round House
When She Woke
Mr. Penumbra's 24-Hour Bookstore: A Novel
The Elegance of the Hedgehog
The Bungalow: A Novel
The Language of Flowers
Juliet, Naked

# 4. Self-Attention model

As for the last model, I use the self-attention block to compute the user embedding from the items in the history. The attention logits are computed from item embeddings + positional encoding. The logits are then normalized using softmax.

The final user vector is computed as the weighted sum of the attention weights and item embeddings. In contrast, the previous model just takes the average of the item embeddings.

I leave the optimization procedure from the previous model unchanged.

In [42]:
embedding_dim = 64
hidden_size = 64
num_heads = 1
lr = 0.005

gpu_options = tf.GPUOptions(allow_growth=True)

sess_sa = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
model_sa = modeling.Model_SA(item_count, embedding_dim, hidden_size, batch_size, num_heads, maxlen)

sess_sa.run(tf.global_variables_initializer())
sess_sa.run(tf.local_variables_initializer())



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [43]:
def sa_item_embs_fn():
  return model_sa.output_item(sess_sa)

In [44]:
def sa_user_embs_fn(hist_item_list, hist_mask_list):
  user_embs = model_sa.output_user(sess_sa, [hist_item_list, hist_mask_list])
  user_embs = np.squeeze(user_embs, axis=1)
  return user_embs

In [45]:
start_time = time.time()

max_iters = 20000
test_iter = 1000
valid_best_recall = 0.0
valid_best_iter = 0
train_data_iter = iter(train_data)

loss_sum = 0.0
for iters in range(1, max_iters + 1):
  src, tgt = next(train_data_iter)
  (user_id_list, item_id_list), (hist_item_list, hist_mask_list) = src, tgt
  loss = model_sa.train(sess_sa, [user_id_list, item_id_list, hist_item_list, hist_mask_list, lr])
  loss_sum += loss

  if iters % test_iter == 0:
    test_time = time.time()
    valid_metrics = eval_model_on(
        data_iter=valid_data,
        item_embs_fn=sa_item_embs_fn,
        user_embs_fn=sa_user_embs_fn)
    print(f"iters: {iters}")
    print(f"train loss: {loss_sum / test_iter}")
    print(f"valid metrics: {valid_metrics}")
    print(f"time spent so far: {((time.time() - start_time) / 60.0):.3f}m")
    loss_sum = 0.0
    
    valid_recall = valid_metrics["recall"]
    if valid_recall > valid_best_recall:
      valid_best_recall = valid_recall
      valid_best_iter = iters

    if iters - valid_best_iter >= 5 * test_iter:
      print('early stopping criterion is met')
      break

iters: 1000
train loss: 7.84919075345993
valid metrics: {'users': 60367, 'recall': 0.0309384783144264, 'ndcg': 0.013233770930525112, 'hitrate': 0.06425696158497192}
time spent so far: 1.504m
iters: 2000
train loss: 7.3851932592391965
valid metrics: {'users': 60367, 'recall': 0.045020746855386, 'ndcg': 0.02030191875047928, 'hitrate': 0.09346165951596071}
time spent so far: 3.008m
iters: 3000
train loss: 6.931842414855957
valid metrics: {'users': 60367, 'recall': 0.056218087322744546, 'ndcg': 0.025252948218746882, 'hitrate': 0.11761392813954644}
time spent so far: 4.514m
iters: 4000
train loss: 6.542975493431092
valid metrics: {'users': 60367, 'recall': 0.06566494773757985, 'ndcg': 0.029282526206027188, 'hitrate': 0.13669720211373765}
time spent so far: 6.020m
iters: 5000
train loss: 6.187354376792908
valid metrics: {'users': 60367, 'recall': 0.06956225495535082, 'ndcg': 0.030931959736520822, 'hitrate': 0.1449136117415144}
time spent so far: 7.526m
iters: 6000
train loss: 5.9121995668411

## SA Quantitative evaluation 

In [46]:
test_metrics = eval_model_on(
    data_iter=test_data,
    item_embs_fn=sa_item_embs_fn,
    user_embs_fn=sa_user_embs_fn)
print(f"test metrics: {test_metrics}")

test metrics: {'users': 60367, 'recall': 0.08604631125961366, 'ndcg': 0.0354178386347241, 'hitrate': 0.1791044776119403}


In [47]:
model_metrics['SA'] = test_metrics

## SA Qualitative evaluation

In [48]:
eval_similar_items(sa_item_embs_fn)

Item title: Gone Girl
Similar items are:
26.216997 Gone Girl
20.250338 The Husband's Secret
19.377922 the rosie project
18.961306 Where'd You Go, Bernadette: A Novel
18.870838 The Goldfinch: A Novel (Pulitzer Prize for Fiction)
18.327732 Me Before You: A Novel
17.937778 Dark Places: A Novel
17.414421 What Alice Forgot
17.31912 The Cuckoo's Calling
17.267544 Hidden

Item title: The Hunger Games (The Hunger Games, Book 1)
Similar items are:
25.331823 The Hunger Games (The Hunger Games, Book 1)
22.63208 Mockingjay (The Final Book of The Hunger Games)
21.524817 Catching Fire (The Hunger Games, Book 2)
20.793644 Catching Fire (The Hunger Games, Book 2)
17.70578 The Call Of The Wild
17.592798 Abraham Lincoln: Vampire Hunter
17.579369 Fancy Nancy: Stellar Stargazer!
16.325891 Insurgent (Divergent)
16.270643 Saved by the Shell! (Teenage Mutant Ninja Turtles) (Pictureback(R))
16.105295 The Host

Item title: The Book Thief
Similar items are:
27.633823 The Book Thief
21.063713 Unbroken
20.127247 

Next, I evaluate the recommendations for the users and how the attention affects the user embedding in the Self-Attention model.

In [49]:
def eval_user_recommendations_with_attention():
  item_embs = sa_item_embs_fn()
  gpu_index = build_item_index(item_embs)
  
  item_att_w, user_embs, = sess_sa.run([model_sa.item_att_w, model_sa.user_eb], feed_dict={
      model_sa.mid_his_batch_ph: eval_hist_item_list,
      model_sa.mask: eval_hist_mask_list,
  })
  item_att_w = np.squeeze(item_att_w, axis=1)
  user_embs = np.squeeze(user_embs, axis=1)
  
  D, I = gpu_index.search(user_embs, 10)
  for att, hist_item_list, hist_mask_list, dd, ii in zip(
      item_att_w, eval_hist_item_list, eval_hist_mask_list, D, I):
    print('For the user with the following history the embedding is formed as:')
    for a, item, mask in zip(att, hist_item_list, hist_mask_list):
      if mask == 0.0:
        break
      print(f"[attention={a:.6f}]", item_id_to_title[item])
    print("Recommendations are are:")
    for d, i in zip(dd, ii):
      print(d, item_id_to_title[i])
    print()
      
eval_user_recommendations_with_attention()

For the user with the following history the embedding is formed as:
[attention=0.183582] Gone Girl
[attention=0.268127] The Dog Stars (Vintage Contemporaries)
[attention=0.240535] Fahrenheit 451 Low Price CD
[attention=0.307756] Ready Player One
Recommendations are are:
16.457071 Ready Player One
14.277361 The Dog Stars (Vintage Contemporaries)
14.076849 The Martian: A Novel
13.689842 The Ocean at the End of the Lane: A Novel
13.121575 Wool - Omnibus Edition
12.939957 World War Z: An Oral History of the Zombie War
12.834494 The Casual Vacancy
12.224459 11/22/63 (Thorndike Press Large Print Core Series)
12.196857 Gone Girl
12.15394 The Road

For the user with the following history the embedding is formed as:
[attention=0.025527] One Last Thing Before I Go: A Novel
[attention=0.021458] Sacre Bleu: A Comedy d'Art
[attention=0.016593] Fahrenheit 451 Low Price CD
[attention=0.016550] The Time Keeper
[attention=0.034473] How to Talk to a Widower
[attention=0.012166] The Adventures of Sherloc

# 5. Results

I present the quantitative results of all models together.

In [50]:
for model, metrics in model_metrics.items():
  print(model)
  print(metrics)

BPR
{'users': 60367, 'recall': 0.0793839708971102, 'ndcg': 0.030340466471052048, 'hitrate': 0.157287922209154}
DNN
{'users': 60367, 'recall': 0.06991044218893151, 'ndcg': 0.03219544022900266, 'hitrate': 0.15322941342124008}
SA
{'users': 60367, 'recall': 0.08604631125961366, 'ndcg': 0.0354178386347241, 'hitrate': 0.1791044776119403}
